# Importing Dependencies

In [1]:
#import warnings
#warnings.filterwarnings('ignore')
from termcolor import colored
import os 
from PyPDF2 import PdfReader
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
from termcolor import colored
os.environ["OPENAI_API_KEY"] = #INSERT YOUR KEY HERE#

# Auxiliar functions

In [2]:
# Reads a PDF file and convert into a raw text
def read_PDF(pdf_dir):
    doc_reader = PdfReader(pdf_dir)
    raw_text = ''
    for i, page in enumerate(doc_reader.pages):
        text = page.extract_text()
        if text:
            raw_text += text
    return raw_text

# Splits raw text into smaller chunks with overlap
def split_text(raw_text):
    text_splitter = CharacterTextSplitter(        
        separator = "\n",
        chunk_size = 2500,
        chunk_overlap  = 400, #striding over the text
        length_function = len,
    )
    texts = text_splitter.split_text(raw_text)
    return texts

# Calculates cost and total number of tokens on a list with openai callbacks
def cost_and_tokens(usage_list):  
    total_cost = 0
    total_tokens = 0
    for c in usage_list:
        total_cost += c.total_cost
        total_tokens += c.total_tokens
    return total_cost,total_tokens

# Red printing to assist evaluation
def print_red(text):
    print(colored(text, 'red', attrs=['bold','underline']))

# Prompt Engeneering

In [3]:
prompt_NE = """
Role: Named entities (NE) extractor.
Act as you are an experienced auditor.  
If you don't know the answer, answer with "undefined", don't try to make up an answer.

{context}

Question: {question}

Reply with only the named entity and include no other commentary:"""

prompt_vote = """
Role: An experienced auditor. 
Task: identify the rapporteur's recommendation regarding the approval or disapproval of the municipality's financial statements.
If you don't know the answer, answer with "undefined", don't try to make up an answer.

{context}

Question: {question}

Reply with only the recommendation vote and include no other commentary.
Examples of answers: Approval, Approval with Reservations, Disapproval:"""

prompt_summary = """Role: You are an accounting expert. 
Task: TL;DR/SUMMARY;
Create a concise and comprehensive summary of the provided text relying strictly on the provided text, without including external information and don't include opnions. Craft a summary that is detailed, 
thorough, in-depth, and complex, while maintaining clarity and conciseness."
{context}
Question: {question}

Reply with only the summarization, do not use pening sentence and include no other commentary."""

prompt_inconsistences = """Role: You are an accounting expert. 
Task: Make a concise and comprehensiveb list with all found inconsistence on the audit report.
Do not include title or opening sentence before the list.
Answer with only a bulletpoint list

{context}

Question: {question}

Present with the list alone and no title."""

prompt_NE_extraction = PromptTemplate(template=prompt_NE, input_variables=['context', 'question'])
prompt_vote_extraction = PromptTemplate(template=prompt_vote, input_variables=['context', 'question'])
prompt_summary_extraction = PromptTemplate(template=prompt_summary, input_variables=['context', 'question'])
prompt_inconsistences_extraction = PromptTemplate(template=prompt_inconsistences, input_variables=['context', 'question'])


#  LLM Model

In [4]:
def QA_LLM(query,prompt_input):
    
    QA = RetrievalQA.from_chain_type(llm=OpenAI(model_name='gpt-3.5-turbo',#cost of $0.0015 / 1K tokens
                                                temperature = 0), 
                                      chain_type="stuff", 
                                      retriever=retriever,
                                      #verbose=True,
                                      chain_type_kwargs={"prompt": prompt_input},
                                      return_source_documents=True)
    with get_openai_callback() as cost:
        answer = QA(query,prompt_input)     
    return answer, cost

# Chatting with PDFs

In [5]:
directory = r"C:\Users\felip\OneDrive - Fundacao Getulio Vargas - FGV\!_Paper04_Chat_PDF\TCE_PIAUI\VOTREL"
files = os.listdir(directory)
pdfs = [file for file in files if file.lower().endswith('.pdf')]
print(len(pdfs),' PDFs were found!\n\n')

732  PDFs were found!




In [6]:
file_list = []
mayor_list = []
municipality_list = []
fiscal_year_list = []
process_number_list = []
rapporteur_list = []
summary_list = []
summary2_list = []
inconsistences_list = []
vote_recommendation_list = []
size_list = []
chunk_list = []
cost_list = []
tokens_list = []

embeddings = OpenAIEmbeddings()

begin_pdf = 221
end_pdf = 230

i=0
for pdf in pdfs:
    
    i+=1
    if (i>=begin_pdf and i<=end_pdf):
        #print('','* '*50,'\n','* '*50,"\n *\t Chating with: ",pdf,'\n','* '*50,'\n','* '*50)
        print('<<',i,'>> ',"CHATTING with: ",pdf,'\n')
        raw_text = read_PDF(directory+'\\'+pdf)
        texts = split_text(raw_text)

        docsearch = FAISS.from_texts(texts, embeddings)
        retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":4})

        cost_pdf = []
        municipality, municipality_cost= QA_LLM("What is the name of the municipality (município)?",prompt_NE_extraction)
        #print('Municipality >>>',municipality['result'],'\n','- '*50)

        mayor, mayor_cost = QA_LLM("What is the name of the mayor (prefeito)?",prompt_NE_extraction)   
        #print('Mayor >>>',mayor['result'],'\n','- '*50)

        fiscal_year, fiscal_year_cost = QA_LLM("What fiscal year or financial year does the document refer to?",prompt_NE_extraction)        
        #print('Fiscal Year >>>',fiscal_year['result'],'\n','- '*50)

        process_number, process_number_cost = QA_LLM("What's the process number of the document?",prompt_NE_extraction)
        #print('Process Number >>>',process_number['result'],'\n','- '*50)

        rapporteur, rapporteur_cost = QA_LLM("What is the name of the rapporteur (conselheiro)?",prompt_NE_extraction)
        #print('rapporteur >>>',rapporteur['result'],'\n','- '*50)

        summary, summary_cost = QA_LLM("Summarize the document for a citizen interested in monitoring public accounts.",prompt_summary_extraction)
        #print('Summary >>>\n',summary['result'],'\n','- '*50)

        summary2, summary2_cost = QA_LLM("Summarize the document's content extracting the key factors in a bullet list.",prompt_summary_extraction)
        #print('Summary2 >>>\n',summary2['result'],'\n','- '*50)

        inconsistences, inconsistences_cost = QA_LLM("Provide a bulletpoint response with inconsistencies found.",prompt_inconsistences_extraction)
        #print('Inconsistences >>>\n',inconsistences['result'],'\n','- '*50)

        vote_recommendation, vote_recommendation_cost = QA_LLM("Whats the rapporteur vote recommendation?",prompt_vote_extraction)
        #print('Vote Recommendation >>>',vote_recommendation['result'],'\n','- '*50)

        cost_pdf = [municipality_cost,mayor_cost,fiscal_year_cost,process_number_cost,rapporteur_cost,summary_cost,summary2_cost,inconsistences_cost,vote_recommendation_cost]
        total_cost,total_tokens = cost_and_tokens(cost_pdf)
        #print('Cost >>>',total_cost,'\n','- '*50)
        print('Tokens >>>',total_tokens,'\n','- '*50)

        #print('Custo:\t',total_cost)
        #print('Tokens:\t',total_tokens)

        cost_list.append(total_cost)
        tokens_list.append(total_tokens)

        file_list.append(pdf)
        size_list.append(len(raw_text))
        chunk_list.append(len(texts))
        municipality_list.append(municipality['result'])
        mayor_list.append(mayor['result'])
        fiscal_year_list.append(fiscal_year['result'])
        process_number_list.append(process_number['result'])
        rapporteur_list.append(rapporteur['result'])
        summary_list.append(summary['result'])
        summary2_list.append(summary2['result'])
        inconsistences_list.append(inconsistences['result'])
        vote_recommendation_list.append(vote_recommendation['result'])
    

<< 221 >>  CHATTING with:  007253_2018_VOTREL - 10812020 - 05112020 - GAB. CONS. JOAQUIM KENNEDY N.BARROS.pdf 



C:\Users\felip\anaconda3\lib\site-packages\langchain\llms\openai.py:179: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\felip\anaconda3\lib\site-packages\langchain\llms\openai.py:753: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Tokens >>> 23880 
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
<< 222 >>  CHATTING with:  009404_2018_VOTREL - 12162021 - 24062021 - GAB. CONS. JOAQUIM KENNEDY N.BARROS.pdf 

Tokens >>> 26749 
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
<< 223 >>  CHATTING with:  009405_2018_VOTREL - 13982022 - 19012022 - AUD. JACKSON VERAS.pdf 

Tokens >>> 26415 
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
<< 224 >>  CHATTING with:  009406_2018_VOTREL - 27702021 - 12052021 - AUD. ALISSON ARAUJO.pdf 

Tokens >>> 13097 
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
<< 225 >>  CHATTING with:  009407_2018_VOTREL - 10942021 - 04062021 - GAB. CONS. KLEBER DANTAS EULALIO.pdf 

Tokens >>> 27690 
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# Save to Dataframe

In [7]:
print('resr')

resr


In [8]:
import pandas as pd

d = {'PDF': file_list,
     'Lenght': size_list[0:35],
     'Number of Chunks': chunk_list[0:35],
     'Municipality': municipality_list, 
     'Mayor': mayor_list,
     'Fiscal Year': fiscal_year_list,
     'Process Number':process_number_list,
     'Rapporteur': rapporteur_list,
     'Vote Recommendation': vote_recommendation_list,
     'Summary': summary_list,
     'Summary List':summary2_list,
     'Inconsistences': inconsistences_list,
     'Cost': cost_list,
     'Number of Tokens': tokens_list
    }


name_excel = 'results_'+str(begin_pdf)+'_'+str(end_pdf)+'.xlsx'

df = pd.DataFrame(data=d)
df.to_excel(name_excel, index=False)

df


,PDF,Lenght,Number of Chunks,Municipality,Mayor,Fiscal Year,Process Number,Rapporteur,Vote Recommendation,Summary,Summary List,Inconsistences,Cost,Number of Tokens
0,007253_2018_VOTREL - 10812020 - 05112020 - GAB...,15221,8,Wall Ferraz,Danilo Araújo Nunes Martins,2017,TC/007253/2018,Joaquim Kennedy Nogueira Barros,Approval with Reservations,The document is a summary of the accounting re...,- The accounting expert recommends the approva...,- Extemporaneous submission of the Budget Bala...,0.036045,23880
1,009404_2018_VOTREL - 12162021 - 24062021 - GAB...,22152,11,Acauã,Reginaldo Raimundo Rodrigues,undefined,TC/009404/2018,Joaquim Kennedy Nogueira Barros,Approval with Reservations,The document is a report on the government's f...,- The document discusses discrepancies in bala...,- Divergências de saldos no Balanço Patrimonia...,0.040439,26749
2,009405_2018_VOTREL - 13982022 - 19012022 - AUD...,17302,9,Agricolândia,Walter Ribeiro Alencar,2018,TC/009405/2018,Jackson Nobre Veras,Disapproval,The document is a report from the Tribunal de ...,- The document discusses the evaluation of the...,- Expenses in the FUNDEB exceeded the received...,0.039883,26415
3,009406_2018_VOTREL - 27702021 - 12052021 - AUD...,3639,2,Água Branca,Jonas Moura de Araújo,2018,009.406/2018,Conselheiro-Substituto Alisson Felipe de Araújo,Approval with Reservations,The document is a proposal for the approval of...,- The document is a proposal for the approval ...,- Divergências entre sagres-contábil e RREO - ...,0.019865,13097
4,009407_2018_VOTREL - 10942021 - 04062021 - GAB...,20953,10,Alagoinha do Piauí,Jorismar José da Rocha,undefined,TC/009407/2018,Kleber Dantas Eulálio,Disapproval,This document is a report on the annual govern...,- The municipality's education rating improved...,- Resultado C (Baixo Nível de Adequação) em vá...,0.041810,27690
5,009408_2018_VOTREL - 11722021 - 14042021 - GAB...,21549,11,Alegrete do Piauí,MÁRCIO WILLIAM MAIA ALENCAR,undefined,TC/009408/2018,WALTÂNIA MARIA N. DE SOUSA LEAL ALVARENGA,Disapproval,The document is a report on the evaluation of ...,- The defense failed to address several issues...,- Late submission of the Multi-Year Plan\n- Op...,0.038018,25077
6,009409_2018_VOTREL - 14942021 - 03082021 - AUD...,22943,11,Alto Longá,HENRIQUE CESAR SARAIVA DE AREA LEÃO COSTA,undefined,TC/009409/2018,CONS. SUBST. JAYLSON FABIANH LOPES CAMPELO,Approval with Reservations,This document is a summary of the findings fro...,- The examined entity published a cash availab...,- Inconsistency in the Demonstrativo da Dispon...,0.043455,28815
7,009411_2018_VOTREL - 13682022 - 10032022 - GAB...,24302,12,Alvorada do Gurguéia,Luis Ribeiro Martins,"2016, 2017, 2018",TC/009411/2018,Cons. Joaquim Kennedy Nogueira Barros,Approval with Reservations,The document is a report on the Government Acc...,- The document is about the audit of the Gover...,- Growth in the percentage of children with ag...,0.040255,26654
8,009412_2018_VOTREL - 15742022 - 10022022 - AUD...,78460,38,Amarante,Diego Lamartine Soares Teixeira,undefined,undefined,DELANO CARNEIRO DA CUNHA CÂMARA,Approval with Reservations,The document discusses the evaluation of the G...,- The government of Brazil has adopted the int...,- Inconsistências das informações prestadas no...,0.038841,25704
9,009412_2018_VOTREL - 15822022 - 14022022 - AUD...,78580,38,Amarante,Diego Lamartine Soares Teixeira,undefined,undefined,DELANO CARNEIRO DA CUNHA CÂMARA,Approval with Reservations,The document discusses the evaluation of gover...,- The government of Brazil has adopted the int...,- Inconsistências das informações prestadas no...,0.040012,26489


In [9]:
df1 = pd.read_excel('total_results_001_220.xlsx')

df_merged = pd.concat([df1,df], ignore_index=True)
df_merged

df_merged.to_excel('total_results_001_230.xlsx', index=False)